In [1]:
import pandas as pd

In [42]:
corp_trans = pd.read_csv('./corp_list.csv')
card_group = pd.read_csv('./playersCardRank.csv')
card = pd.read_csv('./cardDetail.csv').loc[:,['id', 'player', 'name', 'resourceCount','players_terraformRating']]
pre = pd.read_csv('./preprocess.csv')

/Users/huangoushuo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (40,41,42,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
pre

,game_id,seed_game_id,players,generations,createtime,player1,player2,player3,player4,player5,...,randomMAToggle,doubleCorp,initialCorpDraftVariant,pathfindersExpansion,moonStandardProjectVariant,altVenusBoard,escapeVelocityMode,escapeVelocityThreshold,escapeVelocityPeriod,escapeVelocityPenalty
0,f7cbeb6cc687,NaN,4,8,2020-08-31 15:05:21,"{'corporation': 'Septem Tribus', 'playerScore'...","{'corporation': 'Valley Trust (breakthrough)',...","{'corporation': 'Teractor (breakthrough)', 'pl...","{'corporation': 'Poseidon', 'playerScore': 69,...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10d3f58364b6,NaN,2,12,2020-08-31 15:05:21,"{'corporation': 'Aridor', 'playerScore': 144, ...","{'corporation': 'Inventrix (breakthrough)', 'p...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ff2c9f3f3e45,NaN,4,8,2020-08-31 15:05:21,"{'corporation': 'Thorgate (breakthrough)', 'pl...","{'corporation': 'Teractor (breakthrough)', 'pl...","{'corporation': 'Inventrix (breakthrough)', 'p...","{'corporation': 'Mining guild (breakthrough)',...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cf45c3f6faa2,NaN,2,11,2020-08-31 15:05:21,"{'corporation': 'Factorum (breakthrough)', 'pl...","{'corporation': 'Arklight (breakthrough)', 'pl...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,79cef4b6a297,NaN,2,8,2020-08-31 15:05:21,"{'corporation': 'Point Luna', 'playerScore': 6...","{'corporation': 'Celestic', 'playerScore': 43,...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14367,g41f2c9e4c2c5,NaN,2,10,2022-03-10 16:10:05,"{'corporation': 'Valley Trust', 'corporation2'...","{'corporation': 'Recyclon', 'corporation2': ''...",NaN,NaN,NaN,...,NaN,False,True,False,False,False,False,30.0,2.0,1.0
14368,gc97a78f95ac9,NaN,2,10,2022-03-10 16:15:13,"{'corporation': 'Pobolog (breakthrough)', 'cor...","{'corporation': 'Project Workshop', 'corporati...",NaN,NaN,NaN,...,NaN,True,True,True,False,False,False,30.0,2.0,1.0
14369,gec169326022d,NaN,2,11,2022-03-10 16:25:01,"{'corporation': 'Valley Trust (breakthrough)',...","{'corporation': 'Saturn Systems', 'corporation...",NaN,NaN,NaN,...,NaN,True,True,False,False,False,False,30.0,2.0,1.0
14370,g55c6366d7f47,NaN,4,5,2022-03-10 16:46:54,"{'corporation': 'Poseidon', 'corporation2': 'C...","{'corporation': 'Robin Haulings', 'corporation...","{'corporation': 'Lakefront Resorts', 'corporat...","{'corporation': 'Astrodrill', 'corporation2': ...",NaN,...,NaN,True,True,True,False,False,False,30.0,2.0,1.0


In [67]:
def getPlayerNumCorpResult(df, player_num = 4):
    # def expandDoubleCorp(df):
    #     pd.options.mode.chained_assignment = None
    #     df1 = df[df['doubleCorp'] == True]
    #     df2 = df[df['doubleCorp'] == True]
    #     df2['corporation'] = df2['corporation2']
    #     res = pd.concat([df1, df2],axis=0, ignore_index=True)
    #     res['count'] = 0.5
    #     return res
    df = df.loc[df['players'] == player_num].reset_index(drop=True)
    for i in range(1, player_num+1):
        player_idx = 'player'+str(i)
        player_df_pre = df[player_idx].apply(lambda x:eval(x))
        # print(player_idx)
        player_df = pd.json_normalize(player_df_pre).reset_index(drop=True)
        if i == 1:
            res = pd.concat([df,player_df.reindex(df.index)],axis=1)
            print((res.loc[pd.isna(res['player']) == False]).shape[0])
        else:
            mid = pd.concat([df,player_df.reindex(df.index)],axis=1)
            res = pd.concat([res, mid],axis=0, ignore_index=True)
            # print((mid.loc[pd.isna(mid['player']) == False]).shape[0])
        # df = pd.concat([df, pd.json_normalize(df[player_idx])],axis=1)
    res.drop(['player'+str(i) for i in range(1, 7)], axis=1, inplace=True)
    res['count'] = 1
    # res_single = res[~(res['doubleCorp'] == True)]
    # res_double = expandDoubleCorp(res)
    # res_final = pd.concat([res_single, res_double],axis=0, ignore_index=True)
    res_final = res.merge(corp_trans, how='left', on='corporation', suffixes=['','_drop'])
    res_final = res_final.merge(corp_trans, how='left', left_on='corporation2', right_on = 'corporation', suffixes=['','_2'])
    return res_final
corp_df4 = getPlayerNumCorpResult(pre, 4)
corp_df2 = getPlayerNumCorpResult(pre, 2)
corp_df = pd.concat([corp_df4, corp_df2], axis=0).loc[:, ['game_id', 'player', 'players', 'generations', 'createtime', \
                                                        'playerScore', 'position', 'count', 'corporation', 'corporation2', 'cn', 'cn_2', \
                                                        'breakthrough', 'aresExtension', 'erosCardsOption', 'doubleCorp', 'pathfindersExpansion', 'moonExpansion']]
corp_df_group = corp_df.groupby(['cn', 'corporation', 'players']).agg(
    position = ('position', 'mean'),
    playerScore = ('playerScore', 'mean'),
    generations = ('generations', 'mean'),
    total = ('count', 'sum')
).dropna().sort_values('position').reset_index()

4431
5852


In [68]:
corp_df
# card

,game_id,player,players,generations,createtime,playerScore,position,count,corporation,corporation2,cn,cn_2,breakthrough,aresExtension,erosCardsOption,doubleCorp,pathfindersExpansion,moonExpansion
0,f7cbeb6cc687,育亮,4,8,2020-08-31 15:05:21,111,1,1,Septem Tribus,NaN,议会公司,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ff2c9f3f3e45,something,4,8,2020-08-31 15:05:21,88,1,1,Thorgate (breakthrough),NaN,电公司(突破),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39debb03ba72,一直,4,9,2020-08-31 15:05:21,99,1,1,Poseidon,NaN,殖民公司,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b839cdd97347,米饭,4,8,2020-08-31 15:05:21,84,1,1,Point Luna,NaN,点月,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,238d1ff5856f,ender,4,7,2020-08-31 15:05:21,71,1,1,Philares,NaN,贴贴公司,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11699,g676895a3faf8,hitagi,2,12,2022-03-10 14:45:41,141,2,1,Pobolog (breakthrough),StormCraft Incorporated (breakthrough),钛公司(未突破),木星云公司(突破),True,False,False,True,False,False
11700,gdbaae4bed57a,12345,2,10,2022-03-10 15:15:44,111,2,1,Inventrix (breakthrough),Interplanetary Cinematics (breakthrough),科技适应(突破),事件(突破),True,True,True,True,False,False
11701,g41f2c9e4c2c5,LW,2,10,2022-03-10 16:10:05,81,2,1,Recyclon,,铁微生物(未突破),NaN,False,False,False,False,False,False
11702,gc97a78f95ac9,tengwan,2,10,2022-03-10 16:15:13,148,2,1,Project Workshop,Junk Ventures,蓝卡公司,垃圾回收,True,True,True,True,True,False


In [124]:
# 减费
def get4Discount():
    card_list = ['Anti-Gravity Technology', 'Earth Catapult', 'Sky Docks', 'Research Outpost']

    target = card.loc[card['name'].isin(card_list)]

    target['rn'] = target \
                .groupby(['id','player']) \
                .cumcount() + 1
    res = target.query('rn == 4')
    res_final = corp_df.merge(res, left_on=['game_id', 'player'], right_on=['id', 'player'], how='inner', suffixes=['', '_drop'])
    res_final.drop_duplicates(subset=['player'], keep='first', inplace=True)
    res_final['title'] = '黑暗法师'
    res_final['reason'] = '在' + res_final['generations'].astype(str).str[:] + '时代内同时打出所有全卡减费 (4/4)'
    res_final = res_final.sort_values('playerScore', ascending=False).reset_index(drop=True).reset_index(drop=False)
    return res_final
df_get4Discount = get4Discount()


<ipython-input-124-97bf8e6f12da>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['rn'] = target \


In [107]:
# 微生物
def getMicro():
    card_list = ['Ants', 'Viral Enhancers', 'Decomposers', 'Sulphur-Eating Bacteria', 'Extreme-Cold Fungus', 'Venusian Insects', 'GMO Contract', 'Topsoil Contract', 'Bactoviral Research', 'Hydrothermal Vent Archaea', 'Bio-Fertilizer Facility', 'Ecological Survey']

    target = card.loc[card['name'].isin(card_list)]

    target['rn'] = target \
                .groupby(['id','player']) \
                .cumcount() + 1
    target = target.sort_values('rn', ascending=False)
    target.drop_duplicates(subset=['id', 'player'], keep='first', inplace=True)
    res = target.query('rn >= 6')
    res_final = corp_df.merge(res, left_on=['game_id', 'player'], right_on=['id', 'player'], how='inner', suffixes=['', '_drop'])
    res_final.drop_duplicates(subset=['player'], keep='first', inplace=True)
    res_final['title'] = '见微知著'
    res_final['reason'] = '打出微生物核心组件 (' + res_final['rn'].astype(str).str[:] + '/6)'
    res_final = res_final.sort_values('rn', ascending=False).reset_index(drop=True).reset_index(drop=False)
    return res_final
df_getMicro = getMicro()


<ipython-input-107-b0a728b688a2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['rn'] = target \


In [109]:
# 动物
def getAnimal():
    card_list = ['Ecological Zone', 'Viral Enhancers', 'Ecological Zone:ares', 'Ocean Sanctuary', 'Meat Industry', 'GMO Contract', 'Ecological Survey']

    target = card.loc[card['name'].isin(card_list)]

    target['rn'] = target \
                .groupby(['id','player']) \
                .cumcount() + 1
    target = target.sort_values('rn', ascending=False)
    target.drop_duplicates(subset=['id', 'player'], keep='first', inplace=True)
    res = target.query('rn >= 4')
    res_final = corp_df.merge(res, left_on=['game_id', 'player'], right_on=['id', 'player'], how='inner', suffixes=['', '_drop'])
    res_final.drop_duplicates(subset=['player'], keep='first', inplace=True)
    res_final['title'] = '驯兽大师'
    res_final['reason'] = '打出动物核心组件 (' + res_final['rn'].astype(str).str[:] + '/4)'
    res_final = res_final.sort_values('rn', ascending=False).reset_index(drop=True).reset_index(drop=False)
    return res_final
df_getAnimal = getAnimal()

<ipython-input-109-08f82b16d507>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['rn'] = target \


In [111]:
# 木星
def getJovian():
    card_list = ['Water Import From Europa', 'Ganymede Colony', 'Io Mining Industries', 'Terraforming Ganymede']

    target = card.loc[card['name'].isin(card_list)]

    target['rn'] = target \
                .groupby(['id','player']) \
                .cumcount() + 1
    target = target.sort_values('rn', ascending=False)
    target.drop_duplicates(subset=['id', 'player'], keep='first', inplace=True)
    res = target.query('rn >= 3')
    res_final = corp_df.merge(res, left_on=['game_id', 'player'], right_on=['id', 'player'], how='inner', suffixes=['', '_drop'])
    res_final.drop_duplicates(subset=['player'], keep='first', inplace=True)
    res_final['title'] = '宙斯现世'
    res_final['reason'] = '在' + res_final['generations'].astype(str).str[:] + '时代内同时打出所有全卡木星计分 (4/4)'
    res_final = res_final.sort_values('playerScore', ascending=False).reset_index(drop=True).reset_index(drop=False)
    return res_final
df_getJovian = getJovian()

<ipython-input-111-aedf3655f665>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['rn'] = target \


In [94]:
# 事件
def getEvent():
    card_list = ['Media Group', 'Optimal Aerobraking']
    corp_list = ['Interplanetary Cinematics (breakthrough)', 'Interplanetary Cinematics', 'Martian Insurance Group']

    target = card.loc[card['name'].isin(card_list)]

    target['rn'] = target \
                .groupby(['id','player']) \
                .cumcount() + 1
    target = target.sort_values('rn', ascending=False)
    target.drop_duplicates(subset=['id', 'player'], keep='first', inplace=True)
    res = target.query('rn == 2')
    corp_df2 = corp_df.loc[corp_df['corporation'].isin(corp_list) | corp_df['corporation2'].isin(corp_list)]
    res_final = corp_df2.merge(res, left_on=['game_id', 'player'], right_on=['id', 'player'], how='inner', suffixes=['', '_drop'])
    res_final.drop_duplicates(subset=['player'], keep='first', inplace=True)
    res_final['title'] = '西方记者'
    res_final['reason'] = '扮演事件公司并打出所有事件回钱 (2/2)'
    res_final = res_final.sort_values('playerScore', ascending=False).reset_index(drop=True).reset_index(drop=False)
    return res_final
df_getEvent = getEvent()

<ipython-input-94-4e3ddfb48bb5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['rn'] = target \


In [114]:
# 时代数小于4且排名前2
def getLeastGen():
    res_final = corp_df.sort_values('generations')
    res_final = res_final.loc[(res_final['generations'] <= 4) & (res_final['position'] <= 2)]
    res_final.drop_duplicates(subset=['player'], keep='first', inplace=True)
    res_final['title'] = '改造之星'
    res_final['reason'] = '在4时代之前结束游戏并获得前2名 (' + res_final['generations'].astype(str).str[:] + '/4)'
    res_final = res_final.sort_values('playerScore', ascending=False).reset_index(drop=True).reset_index(drop=False)
    return res_final
df_getLeastGen = getLeastGen()

In [116]:
# 时代数小于6且得分>100
def get6GenScore():
    res_final = corp_df.sort_values('playerScore', ascending=False)
    res_final = res_final.loc[(res_final['generations'] <= 6) & (res_final['playerScore'] >= 100)]
    res_final.drop_duplicates(subset=['player'], keep='first', inplace=True)
    res_final['title'] = '百分灵能'
    res_final['reason'] = '在6时代之前结束游戏并获得分数 (' + res_final['playerScore'].astype(str).str[:] + '/100)'
    res_final = res_final.sort_values('playerScore', ascending=False).reset_index(drop=True).reset_index(drop=False)
    return res_final
df_get6GenScore = get6GenScore()

In [118]:
# 时代数小于7且得分>150
def get7GenScore():
    res_final = corp_df.sort_values('playerScore', ascending=False)
    res_final = res_final.loc[(res_final['generations'] <= 7) & (res_final['playerScore'] >= 150)]
    res_final.drop_duplicates(subset=['player'], keep='first', inplace=True)
    res_final['title'] = '北斗七星'
    res_final['reason'] = '在7时代之前结束游戏并获得分数 (' + res_final['playerScore'].astype(str).str[:] + '/150)'
    res_final = res_final.sort_values('playerScore', ascending=False).reset_index(drop=True).reset_index(drop=False)
    return res_final
df_get7GenScore = get7GenScore()

In [127]:
df_get4Discount = get4Discount()
df_getMicro = getMicro()
df_getAnimal = getAnimal()
df_getJovian = getJovian()
df_getEvent = getEvent()
df_getLeastGen = getLeastGen()
df_get6GenScore = get6GenScore()
df_get7GenScore = get7GenScore()

final_df = pd.concat(
    [df_get4Discount, \
        df_getMicro, \
        df_getAnimal, \
        df_getJovian, \
        df_getEvent, \
        df_getLeastGen, \
        df_get6GenScore, \
        df_get7GenScore \
        ]
)

<ipython-input-124-97bf8e6f12da>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['rn'] = target \
<ipython-input-94-4e3ddfb48bb5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['rn'] = target \


In [129]:
final_df.to_csv('./成就.csv', index=False)